#### Code Etabli

In [1]:
import os
import pandas as pd
import numpy as np
import math

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import gc


In [2]:
# Calcule la quantité de Mg à utiliser
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

# Calcule de la longueur du fil fourré et de la masse de fonte limite dans le four de maintien
def calcul_longueur_fil_et_limite_four(
    masse_grappe, nb_moules_heure, masse_fonte_poche, masse_fonte_coulee, pct_mg_fonte_coulee,
    pct_Soufre, tempera_fonte_poche,
    pct_rendement_mg, masse_fil, masse_mg_fil, masse_fonte_coulee_min, masse_fonte_coulee_max, pct_mg_fonte_coulee_min,
    pct_mg_fonte_coulee_max, pct_perdu_mg_coulee_min, pct_perdu_mg_poche_min, temps_traitement, temps_gs,
    heure_lancement):

    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre la fonte minimal (en Kg) dans le four de maintien
    consommation_fonte_min = nb_moules_heure * masse_grappe / 60  # en kg/min
    fonte_four_consommable = masse_fonte_coulee - masse_fonte_coulee_min
    temps_epuis_fonte = fonte_four_consommable/consommation_fonte_min
    delai_avt_traitement_fonte_four = temps_epuis_fonte - temps_traitement



    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre le pourcentage minimal (%) dans le four de maintien
    pct_mg_four_consommable = pct_mg_fonte_coulee - pct_mg_fonte_coulee_min
    temps_epuis_mg = pct_mg_four_consommable/pct_perdu_mg_coulee_min
    delai_avt_traitement_mg_four = temps_epuis_mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de maintien)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)

    # print(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    # mise a jour de la masse fonte avant ajout poche mais après consomation de la fonte
    masse_fonte_four_consommer = (temps_traitement + delai_avt_traitement)*consommation_fonte_min 
    masse_four_limite =  masse_fonte_coulee - masse_fonte_four_consommer


    # mise a jour de la masse mg avant ajout poche après consomation de mg
    pct_mg_four_consommer = (temps_traitement + delai_avt_traitement)*pct_perdu_mg_coulee_min
    pct_mg_four_limite = pct_mg_fonte_coulee - pct_mg_four_consommer


    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( pct_mg_fonte_coulee_max*(masse_four_limite + masse_fonte_poche) - pct_mg_four_limite*masse_four_limite )/ masse_fonte_poche

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    pct_mg_fil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(masse_fonte_poche,pct_Soufre,temps_gs,pct_perdu_mg_poche_min,tempera_fonte_poche,pct_rendement_mg,pct_mg_fil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    longueur_fil_necessaire = Q / (masse_fil* 1e-3)   # en m

    return K,delai_avt_traitement,pct_mg_four_limite,masse_four_limite,longueur_fil_necessaire

def Calcule_Temps_prochain_traitement(delai_avt_traitement,heure_lancement):
    # Trouver les positions des unités de temps dans la chaîne
    pos_hr = heure_lancement.find('hr')
    pos_min = heure_lancement.find('min')
    pos_sec = heure_lancement.find('s')

    # Extraire les valeurs numériques
    now_hours = int(heure_lancement[:pos_hr].strip()) 
    now_minutes = int(heure_lancement[pos_hr+3:pos_min].strip()) 
    now_seconds = int(heure_lancement[pos_min+4:pos_sec].strip()) 


    # Calculer le temps total en secondes
    total_seconds = now_hours * 3600 + now_minutes * 60 + now_seconds + delai_avt_traitement*60

    # Calculer les nouvelles heures, minutes et secondes
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)

    heure_prochain_lancement = f"{hours} hr {minutes} min {seconds} s"
    return heure_prochain_lancement

def export_result(df, dossier_data):
    """
    """
    # Créer le chemin complet du nouveau fichier Excel
    fichier_resultats = os.path.join(dossier_data, 'Resultats.xlsx')

    workbook = Workbook()
    feuille = workbook.active 

    # Écrire le DataFrame dans la feuille
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            feuille.cell(row=r_idx, column=c_idx, value=value)

    # Sauvegarder le classeur
    workbook.save(fichier_resultats)
    workbook.close()
    gc.collect()
    return 

def main_fct(chemin_fichier, dossier_courant):
    # Lecture de la première feuille du fichier Excel
    df = pd.read_excel(chemin_fichier, engine='openpyxl')

    # Supprimer les lignes vides
    df2 = df.dropna(how='all').reset_index(drop=True)

    # Extraction des paramètres Généraux de traitement GS
    pct_rendement_mg, masse_fil, masse_mg_fil, masse_fonte_coulee_min, masse_fonte_coulee_max, pct_mg_fonte_coulee_min= [
        pd.to_numeric(df2.iloc[1, i], errors='coerce') for i in range(0, 6)
    ]

    pct_mg_fonte_coulee_max, pct_perdu_mg_coulee_min, pct_perdu_mg_poche_min, temps_traitement, temps_gs = [
        pd.to_numeric(df2.iloc[1, i], errors='coerce') for i in range(6, 11)
    ]

    heure_lancement = df2.iloc[1, 11]

    # Extraction des variables du Fours de fusion
    pct_Soufre, tempera_fonte_poche = [
        pd.to_numeric(df2.iloc[4, i], errors='coerce') for i in range(5, 7)
    ]

    # Extraction des variables du Fours de couléee
    masse_grappe, nb_moules_heure, masse_fonte_poche, masse_fonte_coulee, pct_mg_fonte_coulee = [
        pd.to_numeric(df2.iloc[7, i], errors='coerce') for i in range(0, 5)
    ]


    # Calcul de la longueur de fil nécessaire
    K,delai_avt_traitement,pct_mg_fonte_coulee_limite,masse_fonte_coulee_limite,longueur_fil_necessaire = calcul_longueur_fil_et_limite_four(
            masse_grappe, nb_moules_heure, masse_fonte_poche, masse_fonte_coulee, pct_mg_fonte_coulee,
        pct_Soufre, tempera_fonte_poche,
        pct_rendement_mg, masse_fil, masse_mg_fil, masse_fonte_coulee_min, masse_fonte_coulee_max, pct_mg_fonte_coulee_min,
        pct_mg_fonte_coulee_max, pct_perdu_mg_coulee_min, pct_perdu_mg_poche_min, temps_traitement, temps_gs,
        heure_lancement)
        
    # Liste des noms de valeurs à rechercher
    output_name = [
        'Pourcentage de magnésium dans le four de coulée au moment de l\'ajout de la poche (%)',
        'Masse de la fonte dans le four de coulée au moment de l\'ajout de la poche (en Kg)',
        'Pourcentage de magnésium dans le four de coulée après  l\'ajout de la poche (%)',
        'Masse de la fonte dans le four de coulée  après l\'ajout de la poche (en Kg)',
        'Temps restant avant le lancement du prochain traitement (en seconde)',
        'Heure de lancement du prochain traitement',
        'Longueur théorique du fil fourré à utiliser (en m)'

    ]

    # Listes pour stocker les résultats
    indices_lignes_output = []
    noms_colonnes_output = []

    # Parcours du DataFrame pour trouver chaque valeur spécifique
    for target_value in output_name:
        found = False
        for index, row in df.iterrows():
            for col_name in df.columns:
                if row[col_name] == target_value:
                    indices_lignes_output.append(index)
                    noms_colonnes_output.append(col_name)
                    found = True
                    break
            if found:
                break


    heure_prochain_lancement = Calcule_Temps_prochain_traitement(delai_avt_traitement,heure_lancement)
    df_res = df.copy()

    masse_mg_coulee = pct_mg_fonte_coulee_limite* masse_fonte_coulee_limite/100  
    masse_mg_poche = K*masse_fonte_poche/100  
    pct_mg_coulee_apres_ajout = (masse_mg_coulee+masse_mg_poche)/(masse_fonte_coulee_limite+masse_fonte_poche)*100
    # df_res.columns.values[10:12] = res_name
    df_res.loc[indices_lignes_output[0] +1, noms_colonnes_output[0]] = pct_mg_fonte_coulee_limite
    df_res.loc[indices_lignes_output[1] +1, noms_colonnes_output[1]] = masse_fonte_coulee_limite
    df_res.loc[indices_lignes_output[2]+1, noms_colonnes_output[2]] = pct_mg_coulee_apres_ajout
    df_res.loc[indices_lignes_output[3]+1, noms_colonnes_output[3]] = masse_fonte_coulee_limite+masse_fonte_poche
    df_res.loc[indices_lignes_output[4]+1, noms_colonnes_output[4]] = delai_avt_traitement
    df_res.loc[indices_lignes_output[5]+1, noms_colonnes_output[5]] = heure_prochain_lancement
    df_res.loc[indices_lignes_output[6]+1, noms_colonnes_output[6]] = longueur_fil_necessaire


    
    export_result(df_res, dossier_courant )
    return 


In [3]:

if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'Sphérodisation.xlsm')
    # On recupere le chemin du dossier data
    dossier_courant = os.path.dirname(chemin_fichier)
    # Solve problème
    main_fct(chemin_fichier, dossier_courant)


def PendantConsommation ( Mg, PFC):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # On produit les moules !! 
    masse_grappe_i = liste_masses_grappes_moules[0]
    cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
    quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
    
    # La cadence de consommation en kg/min du i-ème modèle
    cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
    
    # Quantités de moules du i-ème modèle produits en une minute
    cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min

    quantite_moules_restants = quantite_moules_a_produire_i - cadence_moule_i


    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC



    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat_suivant = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat_suivant = "etat_Fin"

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat_suivant= "etat_Consom"
    
    return Mg, PFC, etat_suivant


#### Variables Globales 

In [1]:
#### Variables Globales 


# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg

temps_serie = 5  # en min
temps_traitement = 10 # en min
temps_serie_courant = temps_serie
temps_gs = 4 # en min


Mgmax, Mgmin, Mglimite = 0.045, 0.035, 0.03
AjoutPoche, DemandePoche = False, False

# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg


#### Fonction Globales 

In [2]:
# Pour calculer la longueur du fil fourré 
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

def calcul_longueur_fil( PPT, S, TPT,
    R, masse_fil, masse_mg_fil, Mgmax, eP, 
    PFClimite, Mglimite ):

    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( Mgmax*(PFClimite + PPT) - Mglimite*PFClimite )/ PPT

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    Mgfil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(PPT,S,temps_gs,eP,TPT,R,Mgfil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    L = Q / (masse_fil* 1e-3)   # en m

    return K, L




# Pour le passage d'un etat à l'autre (consom, Serie, Panne, Fin)
def PendantChangement_serie ( Mg, PFC) :
    # Variables dans ce programme
    global temps_serie, temps_serie_courant

    # pendant Temps_Serie :
    Mg -= eC # perte de Mg par minute ou par seconde
    temps_serie_courant -= 1 # pas de temps en minute
    
    etat_suivant = "etat_Serie"
    if temps_serie_courant == 0:
        temps_serie_courant = temps_serie  # Réinitialiser le temps de série
        
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant

def PendantConsommation ( Mg, PFC):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # On produit les moules !! 
    masse_grappe_i = liste_masses_grappes_moules[0]
    cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
    quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
    
    # La cadence de consommation en kg/min du i-ème modèle
    cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
    

    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC


    # On reste dans l'état consom si il reste des moulles à faire
    etat_suivant= "etat_Consom"

    # Si on a fini de tout produire alors on stoppe la procédure 
    # On passe à l'état Fin
    if not liste_masses_grappes_moules:
        etat_suivant = "etat_Fin"
    
    return Mg, PFC, etat_suivant

def PendantPanne ( Mg, PFC) :
    # Variables dans ce programme
    global Panne

    # pendant Temps_Panne non définis :
    Mg -= eC # perte de Mg par minute ou par seconde


    etat_suivant = "etat_Panne"
    if not Panne :
        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant



def gerer_etat(Mg, PFC, etat_courant):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global running


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        Mg, PFC, etat_suivant = PendantConsommation ( Mg, PFC)

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        Mg, PFC, etat_suivant = PendantChangement_serie ( Mg, PFC)

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg, PFC, etat_suivant = PendantPanne ( Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        running = False  # Fin du processus
        etat_suivant = "etat_Fin"
    return Mg, PFC, etat_suivant


def gerer_poche(Mg, PFC):
    """
    Gère l'ajout de la poche. Si Poche est True, met à jour Mg et PFC.
    
    Args:
    Mg (float): La quantité actuelle de Mg.
    PFC (float): La quantité actuelle de Fonte.

    
    Returns:
    Mg (float): La nouvelle quantité de Mg.
    PFC (float): La nouvelle quantité de fonte.
    """

    # Constantes dans ce programme
    global Mgmax, poche_i
    # Variables dans ce programme
    global AjoutPoche

    if AjoutPoche:
        Mg = Mgmax
        PFC += poche_i
        AjoutPoche = False
    return Mg, PFC




In [3]:
# Pour calculer le temps du prochain Traitement GS
def Calcule_Temps_prochain_traitement(delai_avt_traitement,heure_lancement):
    # Trouver les positions des unités de temps dans la chaîne
    pos_hr = heure_lancement.find('hr')
    pos_min = heure_lancement.find('min')
    pos_sec = heure_lancement.find('s')

    # Extraire les valeurs numériques
    now_hours = int(heure_lancement[:pos_hr].strip()) 
    now_minutes = int(heure_lancement[pos_hr+3:pos_min].strip()) 
    now_seconds = int(heure_lancement[pos_min+4:pos_sec].strip()) 


    if delai_avt_traitement < 0 :
        print("Lancer le Traitement GS !")

    # Calculer le temps total en secondes
    total_seconds = now_hours * 3600 + now_minutes * 60 + now_seconds + delai_avt_traitement*60

    # Calculer les nouvelles heures, minutes et secondes
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)

    heure_prochain_lancement = f"{hours} hr {minutes} min {seconds} s"
    return heure_prochain_lancement    


# Calcule du temps limite avant lancement prochain traitement
def calculer_temps_epuis_PFC(PFC):
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire, temps_serie

    # Calcul de la fonte four consommable
    PFCconsommable = PFC - PFCmin

    # Initialisation du temps total
    temps_total = 0

    # Boucle sur chaque modèle de moules
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min
        
        # Fonte nécessaire pour produire tous les moules de ce modèle
        fonte_necessaire_i = quantite_moules_a_produire_i * masse_grappe_i

        if PFCconsommable >= fonte_necessaire_i:
            # Si la fonte est suffisante pour produire tous les moules
            temps_pour_produire_i = fonte_necessaire_i / cadence_fonte_i
            temps_total += temps_pour_produire_i
            PFCconsommable -= fonte_necessaire_i

            # Ajouter le temps de la série s'il y a d'autres modèles à produire
            if i < len(liste_masses_grappes_moules) - 1:
                temps_total += temps_serie
        else:
            # Si la fonte n'est pas suffisante, calcul du temps possible avec la fonte restante
            temps_possible = PFCconsommable / cadence_fonte_i
            temps_total += temps_possible
            break  # On arrête la boucle, car il n'y a plus de fonte

    return temps_total

def calcul_temps_limite(PFC, Mg):
    
    global Mgmin, eC, temps_traitement

    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre la fonte minimal (en Kg) dans le four de coulée
    temps_epuis_PFC = calculer_temps_epuis_PFC(PFC)
    delai_avt_traitement_fonte_four = temps_epuis_PFC - temps_traitement


    
    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre le pourcentage minimal (%) dans le four de coulée
    Mgconsommable = Mg - Mgmin
    temps_epuis_Mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_Mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de coulée)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    tempslimite = temps_traitement + delai_avt_traitement
    # mise a jour de la masse mg avant ajout poche après consomation de mg
    Mgconsommer = tempslimite*eC
    Mgconsommer = (temps_traitement + delai_avt_traitement)*eC
    Mglimite = Mg - Mgconsommer



    # Mise à jour de la masse de fonte avant ajout dans le four après consommation pendant
    # temps_traitement + delai_avt_traitement
    PFCconsommer = 0
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min

        # Vérification du temps restant par rapport au temps nécessaire pour produire tous les moules de ce modèle
        temps_pour_produire_i = quantite_moules_a_produire_i / (cadence_fonte_i / masse_grappe_i)
        
        if tempslimite > temps_pour_produire_i:
            # Si le délai est supérieur au temps nécessaire, consommer toute la fonte pour ce modèle
            PFCconsommer += cadence_fonte_i * temps_pour_produire_i
            tempslimite -= temps_pour_produire_i

            if i < len(liste_masses_grappes_moules) - 1:
                tempslimite -= temps_serie
        else:
            # Sinon, consommer la fonte en fonction du temps restant
            PFCconsommer += cadence_fonte_i * tempslimite
            break

    PFClimite =  PFC - PFCconsommer


    return delai_avt_traitement, Mglimite, PFClimite




# def gerer_Prevention(Mg, PFC, etat_courant):

#     delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(PFC, Mg)
#     if delai_avt_traitement == 0 :

#     return Mg, PFC, etat_suivant



In [4]:
import plotly.graph_objs as go

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")

    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

fig1, scatter1, fig2, scatter2 = create_figures()

#### Fonctions interactives 

##### Sans bonnes couleurs 

In [7]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import time


def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    global Poche, poche_i

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"  
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")

    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC
    global Poche, poche_i, Mgmax
    
    while running:
        # Mettre à jour les données de temps
        timedata.append(t)
        
        # Appeler la fonction pour gérer l'état
        Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)

        # Si l'on ajoute de la poche
        Mg, PFC = gerer_poche(Mg, PFC)

        
        # Mettre à jour les données pour les graphiques
        Mgdata.append(Mg)
        PFCdata.append(PFC)

        with fig1.batch_update():
            scatter1.update(x=timedata, y=Mgdata)
        
        with fig2.batch_update():
            scatter2.update(x=timedata, y=PFCdata)
            
        t += time_step
        etat_courant = etat_suivant
        
        # Pause pour simuler le temps entre les mises à jour
        time.sleep(time_step)

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"


    # # Démarrer la mise à jour des données dans un nouveau thread ou processus
    import threading
    thread = threading.Thread(target=update_data)
    thread.start()


def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    with fig1.batch_update():
        scatter1.update(x=timedata, y=Mgdata)
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"


def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"

def poche_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Poche
    Poche = True
    etat_courant = "etat_Consom"


def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False


def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    poche_button = widgets.Button(description="Poche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")
    poche_button = widgets.Button(description="Poche")



    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    poche_button.on_click(poche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)


    display(widgets.HBox([start_button, reset_button, stop_button, poche_button, serie_button, panne_button]), fig1, fig2)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig1, scatter1, fig2, scatter2
    fig1, scatter1, fig2, scatter2 = create_figures()
    create_buttons()

# Appel de la fonction principale pour démarrer l'application
main()

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '990bc82b-525b-49e6-b94e-c159ec88fc8d'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '5e2b56c4-e2e2-40c9-ba57-ff186f821f7d'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

##### Avec bonnes couleurs 

In [10]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import time
from threading import Timer

segments = []


message_output = widgets.Output()

def initialize_message():
    """Initialise le widget Output avec un fond blanc et pas de message."""
    with message_output:
        message_output.clear_output()
        display(widgets.HTML(
            "<div style='background-color:white; color:black; font-size:20px; padding:10px; text-align:center;'>"
            "Aucun message</div>"
        ))

def update_message(message):
    """Met à jour le message affiché avec un style personnalisé."""
    with message_output:
        message_output.clear_output()  # Efface les anciens messages
        display(widgets.HTML(
            f"<div style='background-color:yellow; color:red; font-size:24px; font-weight:bold; "
            f"border:2px solid black; padding:10px; text-align:center;'>"
            f"{message}</div>"
        ))

def Demandepoche(PFC, Mg) :
    global DemandePoche, AjoutPoche

    if DemandePoche :
        delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(PFC, Mg)
        message = f"Lancer la poche dans {delai_avt_traitement} minutes"
        update_message(message)
        if AjoutPoche :
            DemandePoche = False
    return


def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()

    # Un seul segment pour Mg, avec la couleur dynamique
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")
    
    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

import asyncio


# async def update_data():
def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC
    global Mgmax, Mgmin, Mglimite
    global segments

    while running:

        if t == 0 :
            timedata.append(0)
            Mgdata.append(0.045)
        else :
            Demandepoche(PFC, Mg)
            Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)
            Mg, PFC = gerer_poche(Mg, PFC)
            timedata.append(t)
            etat_courant = etat_suivant
            Mgdata.append(Mg)
            PFCdata.append(PFC)

            with fig2.batch_update():
                scatter2.update(x=timedata, y=PFCdata)

            x0, x1 = timedata[-2], timedata[-1]
            y0, y1 = Mgdata[-2], Mgdata[-1]

            if y0 > Mgmin and y1 > Mgmin:
                color = 'green'
            elif y0 < Mglimite and y1 < Mglimite:
                color = 'red'
            else:
                color = 'orange'

            segments.append(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines+markers', line=dict(color=color), showlegend=False))

            with fig1.batch_update():
                fig1.data = []  # Efface les anciennes données
                fig1.add_traces(segments)
                

        t += time_step
        # Pause pour simuler le temps entre les mises à jour
        time.sleep(time_step)
        # await asyncio.sleep(time_step)
        # Timer(time_step, update_data).start()

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"

    # Démarrer la mise à jour des données
    # update_data()
    
    # asyncio.ensure_future(update_data())

    # # Démarrer la mise à jour des données dans un nouveau thread ou processus
    import threading
    thread = threading.Thread(target=update_data)
    thread.start()


def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    global segments
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    initialize_message()
    with fig1.batch_update():
        fig1.data = []  # Efface les anciennes données
        segments = []
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"

def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"


def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def Demandepoche_plot(button):
    """Fonction pour passer à l'état de poche."""
    global DemandePoche
    DemandePoche = True
    # update_message("Quand lancer la prochain poche ?")


def Ajoutpoche_plot(button):
    """Fonction pour passer à l'état de poche."""
    global AjoutPoche
    AjoutPoche = True
    # update_message("AJOUT DE LA POCHE")

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    Demandepoche_button = widgets.Button(description="DemandePoche")
    Ajoutpoche_button = widgets.Button(description="AjoutPoche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")

    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    Ajoutpoche_button.on_click(Ajoutpoche_plot)
    Demandepoche_button.on_click(Demandepoche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)

    display(widgets.HBox([start_button, reset_button, stop_button, Demandepoche_button, Ajoutpoche_button, serie_button, panne_button]),message_output, 
            fig1, fig2)


def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig1, scatter1, fig2, scatter2
    fig1, scatter1, fig2, scatter2 = create_figures()
    create_buttons()
    initialize_message()

# Appel de la fonction principale pour démarrer l'application
main()

Output()

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '70436d99-56af-4495-b659-6eef70ea4e9e'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'ee2ed1a4-d1b3-4a35-a8fd-18a1c99bd2b7'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

#### Test lecture

##### Autres

In [17]:

# écris un code qui lis sans arrêt dans un fichier txt pour effectuer des instructions
# Ce fichier peux être changer par l'utilisateur régulièrement




# Ok, merci. Maintenant on veut affiche en fonction de ce que l'on lit dans le fichier. Si l'on lit Option1 alors on affiche sin(t) , si l'on lit Option2 alors 2+t. 
# Le temps t varie a chaque fonction dans la boucle principale. On veut une unique figure dans tout le code. A chaque changement d'option, on calcule sin(t) ou 2+t et l'on rajoute dans la liste des données à afficher. On veut avoir un affichage continue dans une unique figure et graphique qui est mise à jour constamment  avec plotly.


# un peu comme ceci 

# def create_figures():
#     """Crée et configure les figures Plotly pour Mg et PFC."""
#     fig1 = go.FigureWidget()

#     # Un seul segment pour Mg, avec la couleur dynamique
#     scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    
#     fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")
    
#     fig2 = go.FigureWidget()
#     scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
#     fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

#     return fig1, scatter1, fig2, scatter2


# update_scatter(scatter, x_data, y_data) ne marche pas non plus


# Toujours aucune est tracer, peux- tu m'expliquer les raisons éventuels ?


# En rajoutant fig.show() après mise à jour, on a plusieurs figure crée, or  je souhaite conserver une unique figure !!




##### Parametres Globales à mettre dans fichier

In [3]:

# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg

temps_serie = 5  # en min
temps_traitement = 10 # en min
temps_serie_courant = temps_serie
temps_gs = 4 # en min


Mgmax, Mgmin, Mglimite = 0.045, 0.035, 0.03
AjoutPoche, DemandePoche = False, False

# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

##### Fonctions pour concernant les 4 etat du FOUR de COULEE

In [ ]:
# Pour le passage d'un etat à l'autre (consom, Serie, Panne, Fin)
def PendantChangement_serie ( Mg, PFC) :
    # Variables dans ce programme
    global temps_serie, temps_serie_courant

    # pendant Temps_Serie :
    Mg -= eC # perte de Mg par minute ou par seconde
    temps_serie_courant -= 1 # pas de temps en minute
    
    etat_suivant = "etat_Serie"
    if temps_serie_courant == 0:
        temps_serie_courant = temps_serie  # Réinitialiser le temps de série
        
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant

def PendantConsommation ( Mg, PFC):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # On produit les moules !! 
    masse_grappe_i = liste_masses_grappes_moules[0]
    cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
    quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
    
    # La cadence de consommation en kg/min du i-ème modèle
    cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
    

    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC


    # On reste dans l'état consom si il reste des moulles à faire
    etat_suivant= "etat_Consom"

    # Si on a fini de tout produire alors on stoppe la procédure 
    # On passe à l'état Fin
    if not liste_masses_grappes_moules:
        etat_suivant = "etat_Fin"
    
    return Mg, PFC, etat_suivant

def PendantPanne ( Mg, PFC) :
    # Variables dans ce programme
    global Panne

    # pendant Temps_Panne non définis :
    Mg -= eC # perte de Mg par minute ou par seconde


    etat_suivant = "etat_Panne"
    if not Panne :
        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant





def gerer_etat(Mg, PFC, etat_courant):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global running


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        Mg, PFC, etat_suivant = PendantConsommation ( Mg, PFC)

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        Mg, PFC, etat_suivant = PendantChangement_serie ( Mg, PFC)

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg, PFC, etat_suivant = PendantPanne ( Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        running = False  # Fin du processus
        etat_suivant = "etat_Fin"
    return Mg, PFC, etat_suivant



##### Fonctions pour concernant la gestion des poches

In [18]:

import ipywidgets as widgets

message_output = widgets.Output()

def initialize_message():
    """Initialise le widget Output avec un fond blanc et pas de message."""
    with message_output:
        message_output.clear_output()
        display(widgets.HTML(
            "<div style='background-color:white; color:black; font-size:20px; padding:10px; text-align:center;'>"
            "Aucun message</div>"
        ))

def update_message(message):
    """Met à jour le message affiché avec un style personnalisé."""
    with message_output:
        message_output.clear_output()  # Efface les anciens messages
        display(widgets.HTML(
            f"<div style='background-color:yellow; color:red; font-size:24px; font-weight:bold; "
            f"border:2px solid black; padding:10px; text-align:center;'>"
            f"{message}</div>"
        ))

def PendantReset () :
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_suivant, Mg, PFC
    global segments
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_suivant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    initialize_message()
    with fig1.batch_update():
        fig1.data = []  # Efface les anciennes données
        segments = []
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)
    
    return 

def PendantInitialisation () :
    """Fonction pour initialiser les données et le temps t."""
    global running, time_step, timedata, PFCdata, Mgdata
    global etat_courant, Mg, PFC, t
    global segments
    global DemandePoche, AjoutPoche, Panne
    timedata = []
    PFCdata = []
    Mgdata = []
    # etat_courant, etat_suivant = "etat_Consom", "etat_Consom"
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC
    t = 0

    
    return 








# Pour calculer la longueur du fil fourré 
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

def calcul_longueur_fil( PPT, S, TPT,
    R, masse_fil, masse_mg_fil, Mgmax, eP, 
    PFClimite, Mglimite ):

    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( Mgmax*(PFClimite + PPT) - Mglimite*PFClimite )/ PPT

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    Mgfil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(PPT,S,temps_gs,eP,TPT,R,Mgfil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    L = Q / (masse_fil* 1e-3)   # en m

    return K, L


# Calcule du temps limite avant lancement prochain traitement
def calculer_temps_epuis_PFC(PFC):
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire, temps_serie

    # Calcul de la fonte four consommable
    PFCconsommable = PFC - PFCmin

    # Initialisation du temps total
    temps_total = 0

    # Boucle sur chaque modèle de moules
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min
        
        # Fonte nécessaire pour produire tous les moules de ce modèle
        fonte_necessaire_i = quantite_moules_a_produire_i * masse_grappe_i

        if PFCconsommable >= fonte_necessaire_i:
            # Si la fonte est suffisante pour produire tous les moules
            temps_pour_produire_i = fonte_necessaire_i / cadence_fonte_i
            temps_total += temps_pour_produire_i
            PFCconsommable -= fonte_necessaire_i

            # Ajouter le temps de la série s'il y a d'autres modèles à produire
            if i < len(liste_masses_grappes_moules) - 1:
                temps_total += temps_serie
        else:
            # Si la fonte n'est pas suffisante, calcul du temps possible avec la fonte restante
            temps_possible = PFCconsommable / cadence_fonte_i
            temps_total += temps_possible
            break  # On arrête la boucle, car il n'y a plus de fonte

    return temps_total

def calcul_temps_limite(PFC, Mg):
    
    global Mgmin, eC, temps_traitement

    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre la fonte minimal (en Kg) dans le four de coulée
    temps_epuis_PFC = calculer_temps_epuis_PFC(PFC)
    delai_avt_traitement_fonte_four = temps_epuis_PFC - temps_traitement


    
    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre le pourcentage minimal (%) dans le four de coulée
    Mgconsommable = Mg - Mgmin
    temps_epuis_Mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_Mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de coulée)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    tempslimite = temps_traitement + delai_avt_traitement
    # mise a jour de la masse mg avant ajout poche après consomation de mg
    Mgconsommer = tempslimite*eC
    Mgconsommer = (temps_traitement + delai_avt_traitement)*eC
    Mglimite = Mg - Mgconsommer



    # Mise à jour de la masse de fonte avant ajout dans le four après consommation pendant
    # temps_traitement + delai_avt_traitement
    PFCconsommer = 0
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min

        # Vérification du temps restant par rapport au temps nécessaire pour produire tous les moules de ce modèle
        temps_pour_produire_i = quantite_moules_a_produire_i / (cadence_fonte_i / masse_grappe_i)
        
        if tempslimite > temps_pour_produire_i:
            # Si le délai est supérieur au temps nécessaire, consommer toute la fonte pour ce modèle
            PFCconsommer += cadence_fonte_i * temps_pour_produire_i
            tempslimite -= temps_pour_produire_i

            if i < len(liste_masses_grappes_moules) - 1:
                tempslimite -= temps_serie
        else:
            # Sinon, consommer la fonte en fonction du temps restant
            PFCconsommer += cadence_fonte_i * tempslimite
            break

    PFClimite =  PFC - PFCconsommer


    return delai_avt_traitement, Mglimite, PFClimite





def Demandepoche(PFC, Mg) :
    global DemandePoche, AjoutPoche

    if DemandePoche :
        delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(PFC, Mg)
        message = f"Lancer la poche dans {delai_avt_traitement} minutes"
        update_message(message)
        if AjoutPoche :
            DemandePoche = False
    return



def gerer_poche(Mg, PFC):
    """
    Gère l'ajout de la poche. Si Poche est True, met à jour Mg et PFC.
    
    Args:
    Mg (float): La quantité actuelle de Mg.
    PFC (float): La quantité actuelle de Fonte.

    
    Returns:
    Mg (float): La nouvelle quantité de Mg.
    PFC (float): La nouvelle quantité de fonte.
    """

    # Constantes dans ce programme
    global Mgmax, poche_i
    # Variables dans ce programme
    global AjoutPoche

    if AjoutPoche:
        Mg = Mgmax
        PFC += poche_i
        AjoutPoche = False
    return Mg, PFC




In [30]:
import numpy as np
import plotly.graph_objs as go
import time
import os
from IPython.display import display


# Fichier : Gestion de fichier
def read_file(filepath):
    """Lit le fichier et retourne son contenu."""
    with open(filepath, 'r') as file:
        return file.read().strip()


# Graphiques : Création et mise à jour des figures
def create_plot(title, xaxis_title, yaxis_title):
    """Crée un graphique Plotly avec des axes configurés."""
    fig = go.FigureWidget()
    scatter = fig.add_scatter(mode='lines+markers').data[0]
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title)
    display(fig)
    return fig, scatter


def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig_Mg, scatter_Mg = create_plot("Consommation de Mg", "Temps", "Mg")
    fig_PFC, scatter_PFC = create_plot("Consommation de Fonte", "Temps", "Fonte")
    return fig_Mg, scatter_Mg, fig_PFC, scatter_PFC


# Pour le PFC
def update_plot_data(fig, scatter, xdata, ydata):
    """Met à jour les données d'un graphique Plotly."""
    with fig.batch_update():
        scatter.update(x=xdata, y=ydata)


# Pour le Mg
def add_colored_segment(fig, x0, x1, y0, y1, color):
    """Ajoute un segment coloré à un graphique."""
    segment = go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines+markers', line=dict(color=color), showlegend=False)
    with fig.batch_update():
        # fig.data = []  # Efface les anciennes données
        fig.add_trace(segment)

def determine_segment_color(y0, y1, Mgmin, Mglimite):
    """Détermine la couleur du segment en fonction des valeurs de Mg."""
    if y0 > Mgmin and y1 > Mgmin:
        return 'green'
    elif y0 < Mglimite and y1 < Mglimite:
        return 'red'
    else:
        return 'orange'
    

# Pour Mise à jour régulière de la figure
def update_figure(fig1, fig2):
    """Met à jour les données dans les figures Plotly."""
    update_plot_data(fig2, scatter2, timedata, PFCdata)

    x0, x1 = timedata[-2], timedata[-1]
    y0, y1 = Mgdata[-2], Mgdata[-1]

    color = determine_segment_color(y0, y1, Mgmin, Mglimite)
    add_colored_segment(fig1, x0, x1, y0, y1, color)



segments = []
def monitor_file(filepath, fig1, fig2, interval=2):
    """Surveille les changements dans un fichier et met à jour les graphiques en fonction du contenu."""

    global running, time_step, timedata, PFCdata, Mgdata
    global etat_courant, Mg, PFC, t
    global segments
    global DemandePoche, AjoutPoche, Panne
    timedata = []
    PFCdata = []
    Mgdata = []
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC
    t = 0




    last_modified = None
    while etat_courant != "etat_Fin":
        try:
            # Vérifie la date de dernière modification
            current_modified = os.path.getmtime(filepath)
            
            # Si le fichier a été modifié, relit le fichier
            if last_modified is None or current_modified > last_modified:
                last_modified = current_modified
                current_content = read_file(filepath)
                print(f"Fichier mis à jour : {current_content}")




    
                if current_content == "START":
                    running = True

                    Panne = False
                    etat_courant = "etat_Consom"
                
                elif current_content == "STOP":
                    """Fonction pour arrêter la mise à jour."""
                    running = False
                
                elif current_content == "RESET":
                    PendantReset ()
                
                elif current_content == "DEMANDEPOCHE":
                    """Fonction pour passer à l'état de poche."""
                    DemandePoche = True

                elif current_content == "AJOUTPOCHE":
                    """Fonction pour passer à l'état de poche."""
                    AjoutPoche = True

                elif current_content == "SERIE":
                    """Fonction pour passer à l'état de série."""
                    etat_courant = "etat_Serie"          

                elif current_content == "PANNE":
                    """Fonction pour passer à l'état de panne."""
                    Panne = True
                    etat_courant = "etat_Panne"

                elif current_content == "FIN":
                    """Fonction pour passer à l'état de panne."""
                    etat_courant = "etat_Fin"     
                
                else:
                    break

            if running : # Pour pourvoir stopper la simulation
                if t == 0 :
                    timedata.append(0)
                    PFCdata.append(PFC)
                    Mgdata.append(Mg)
                else :
                    Demandepoche(PFC, Mg)
                    Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)
                    Mg, PFC = gerer_poche(Mg, PFC)
                    etat_courant = etat_suivant
                    timedata.append(t)
                    Mgdata.append(Mg)
                    PFCdata.append(PFC)

                # print(etat_courant,etat_suivant)
                print(etat_courant)
                update_figure(fig1, fig2)

                t += 1

        except FileNotFoundError:
            print(f"Le fichier {filepath} n'existe pas.")
        except Exception as e:
            print(f"Une erreur est survenue : {e}")
        
        time.sleep(interval)

if __name__ == "__main__":
    filepath = "mon_fichier.txt"  # Spécifie le chemin du fichier à surveiller
    fig1, scatter1, fig2, scatter2 = create_figures()  # Crée les figures
    monitor_file(filepath, fig1, fig2)  # Met à jour les figures continuellement







# Peux-tu réorganiser ce code pour le rendre plus claire, lisible et facilement modifiable en particulier la fonction monitor_file?


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '5f7388e3-5ccb-40e2-8511-ba2311e2b7b5'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'cad2536a-5dc8-432e-82c6-4dc64f322ea4'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

Fichier mis à jour : START
etat_Consom
Une erreur est survenue : list index out of range
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
Fichier mis à jour : RESET


Fichier mis à jour : START
etat_Consom
Une erreur est survenue : list index out of range
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
Fichier mis à jour : AJOUTPOCHE
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
Fichier mis à jour : PANNE
etat_Panne
etat_Panne
etat_Panne
etat_Panne
etat_Panne
Fichier mis à jour : SERIE
etat_Serie
etat_Serie
etat_Serie
etat_Serie
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
etat_Consom
Fichier mis à jour : AJOUTPOCHE
etat_Consom
etat_Consom
etat_Consom
etat_Consom
Fichier mis à jour : FIN
etat_Fin
